<a href="https://colab.research.google.com/github/LeeSeungwon89/Deep-learning_Theory/blob/main/7-2%20%EC%8B%AC%EC%B8%B5%20%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **7-2 심층 신경망**

인공 신경층에 여러 층을 추가하여 심층 신경망을 생성하겠습니다. 일반 신경망의 성능을 제고하는 작업입니다.

## **2개의 층**

이전 챕터에서 사용했던 패션 MNIST 데이터 세트를 가져오겠습니다. 픽셀 값을 0 ~ 1 사이로 좁히고, 2차원 배열을 1차원 배열로 펼친 후 훈련 세트에서 검증 세트를 분리하겠습니다.

In [21]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

**심층 신경망(deep neural network, DNN)**을 생성하기 위해 기존 신경망 모델에 층 2개를 추가해 보겠습니다. 입력층과 출력층 사이에 새로운 밀집층 2개를 추가하는 것입니다. 추가할 밀집층은 **은닉층(hidden layer)**이라고 부릅니다.

출력층의 활성화 함수는 적용할 수 있는 종류가 제한됩니다. 이진 분류의 경우 시그모이드 함수를 적용하고, 다중 분류의 경우 소프트맥스 함수를 적용합니다. 반면 은닉층의 활성화 함수는 자유롭게 적용할 수 있습니다. 참고로 회귀 문제에서는 활성화 함수를 적용할 필요가 없습니다(`Dense` 클래스의 `activation` 매개변수에 값을 지정하지 않습니다). 회귀 출력은 임의의 숫자이므로 출력층의 선형 방정식에 따라 도출된 값을 그대로 출력하기 때문입니다.

은닉층에 많이 사용하는 함수는 시그모이드 함수입니다. 시그모이드 함수는 유닛의 출력값을 0 ~ 1 사이로 압축합니다. 시그모이드 활성 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 생성해 보겠습니다.

In [22]:
# 신경망의 첫 번째 층(여기서는 은닉층)에는 `input_shape` 매개변수에 입력 크기를 지정합니다.
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
# 출력층을 생성합니다.
dense2 = keras.layers.Dense(10, activation='softmax')

은닉층에 지정한 유닛은 100개지만 특별한 기준을 토대로 지정한 것은 아닙니다. 유닛 개수에 따라 성능이 좌우되므로 적합한 유닛 개수를 지정하는 일에는 상당한 노하우가 필요합니다. 다만 한 가지 조건이 있습니다. 출력층의 유닛보다 많은 유닛을 지정해야 합니다. 출력층의 유닛이 10개인데 은닉층에 이보다 더 적은 유닛을 지정하면 부족한 정보가 전달될 것입니다.

## **심층 신경망 만들기**

`Dense` 클래스로 생성한 두 인스턴스를 사용하여 심층 신경망을 생성해 보겠습니다.

In [23]:
# 여러 층을 추가하려면 리스트에 담아 지정합니다. 단, 출력층은 마지막 원소로 전달합니다. 
model = keras.Sequential([dense1, dense2])

모델의 `summary()` 메서드를 호출하여 각 층에 대한 정보를 확인해 보겠습니다.

In [24]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


은닉층과 출력층이 순차로 나열되었습니다. 은닉층의 출력 크기인 `(None, 100)`에서 첫 번째 차원인 `None`은 샘플 개수를 의미합니다. 아직 샘플 개수가 정의되지 않았으므로 `None`입니다. `fit()` 메서드에 훈련 데이터를 주입하면 이 데이터를 한꺼번에 사용하지 않고 나눠서 수차례에 걸쳐 경사 하강법(**미니배치 경사 하강법**)을 수행합니다. 케라스의 미니배치 크기는 기본적으로 32개이며 `fit()` 메서드의 `batch_size` 매개변수에 지정하여 크기를 변경할 수 있습니다. 따라서 샘플 개수를 `None`으로 두고 어떤 배치 크기에도 유연하게 대응하게 하는 것입니다. 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 **배치 차원**이라고 부릅니다. 두 번째 차원인 `100`은 유닛 개수입니다.

`Param #`은 모델 파라미터 개수를 의미합니다. 각 층의 모델 파라미터 개수에 대한 산출 공식은 아래와 같습니다. 참고로 각 유닛 개수와 절편 개수는 동수입니다.

- `dense`: 입력층 픽셀 784개 x 은닉층 유닛 100개 + 절편 100개 = 78,500개

- `dense_1`: 은닉층 유닛 100개 x 출력층 유닛 10개 x 절편 10개 = 1,010개

총 모델 파라미터 개수와 훈련될 파라미터 개수는 79,510개로 동수입니다. 경사 하강법으로 훈련되지 않는 파라미터를 가진 층도 있으므로 `Non-trainable params`는 0입니다. 

## **층을 추가하는 다른 방법**

층을 추가한 위 방법과 다른 방법도 있습니다. `Sequential` 클래스를 이용하는 방법입니다. 아래 코드로 구현하겠습니다.

In [28]:
# 각 층과 모델에 이름을 부여하기 위해 `name` 매개변수에 값을 지정합니다.
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
    ], name='fashion_MNIST_model')

model.summary()

Model: "fashion_MNIST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


다만 이 방법을 사용하면 여러 층을 추가할 때 클래스 생성자가 길어지고 다른 조건에 따라 층을 추가하기도 어렵습니다. 아래 방법처럼 `add()` 메서드를 사용하는 방법이 가장 널리 사용됩니다.

In [29]:
model = keras.Sequential(name='fashion_MNIST_model')
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

model.summary()

Model: "fashion_MNIST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


모델 훈련을 수행해 보겠습니다.

In [30]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5670 - accuracy: 0.8092
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4077 - accuracy: 0.8518
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3730 - accuracy: 0.8648
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3513 - accuracy: 0.8716
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3351 - accuracy: 0.8785


이전 챕터의 결과보다 높은 정확도가 도출됐습니다. 이렇게 몇 층만 추가해도 성능 향상을 도모할 수 있습니다. 

## **렐루 함수**

**렐루 함수(ReLU function)**는 이미지 분류 문제에서 높은 성능을 낼 수 있는 활성화 함수이며 심층 신경망에서 매우 뛰어납니다. 초창기 인공 신경망의 은닉층에는 시그모이드 함수가 많이 사용됐습니다. 그러나 이 함수는 그래프 상에서 보면 좌우의 끝으로 향할수록 위 아래의 끝으로 붙기 때문에 올바른 출력을 생성하는 데 신속한 대응이 어렵습니다. 특히 심층 신경망일수록 효과가 누적되면서 학습이 더 어려워집니다. 이를 개선할 목적으로 개발된 함수가 렐루 함수입니다. 

렐루 함수는 입력이 양수이면 활성화 함수가 없는 것처럼 입력을 통과시키고, 음수이면 0으로 만듭니다. [링크](https://blog.naver.com/totalcmd/222707125949)에서 확인할 수 있듯이, max(0, z)에서 z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력합니다.

참고로 이미지 분류 문제에 `Flatten` 클래스가 자주 사용됩니다. 이전에는 이미지 파일을 `reshape()` 메서드를 사용하여 1차원으로 펼쳤지만 이 클래스를 사용하면 모든 작업을 한번에 수행합니다. 이 클래스를 층처럼 입력층과 은닉층 사이에 추가하므로 **Flatten 층**이라고 부릅니다. 다만 배치 차원을 제외하고 모든 나머지 입력 차원을 일렬로 펼치는 역할만 수행하므로 입력에 대한 가중치와 절편이 없습니다. 모델 성능에는 기여하지 않습니다. 

Flatten 층을 추가해 보겠습니다. 은닉층에 지정했던 `input_shape` 매개변수를 Flatten 층으로 옮기는 것이 중요합니다. 은닉층에 지정할 활성화 함수는 렐루 함수를 지정하겠습니다.

In [32]:
model = keras.Sequential(name='fashion_MNIST_model')
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu', name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

model.summary()

Model: "fashion_MNIST_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이 신경망은 깊이가 2인 심층 신경망입니다. Flatten 층은 모델 파라미터가 0이므로 학습을 위한 층이 아닙니다. 

Flatten 층을 추가하면 입력값 차원을 짐작하기 좋습니다. 케라스 API는 가능하다면 입력 데이터 전처리 과정을 모델에 전부 포함시키는 것을 추구합니다.

모델을 훈련해 보겠습니다. 이번엔 `reshape()` 메서드를 사용하지 않은 전체 코드를 작성하겠습니다.

In [5]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

model = keras.Sequential(name='fashion_MNIST_model')
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu', name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5315 - accuracy: 0.8122
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3908 - accuracy: 0.8589
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3528 - accuracy: 0.8723
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3319 - accuracy: 0.8815
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3176 - accuracy: 0.8864


성능이 조금 더 향상됐습니다. 에포크를 높여 보겠습니다.

In [3]:
model.fit(train_scaled, train_target, epochs=10)

Epoch 1/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3096 - accuracy: 0.8906
Epoch 2/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2993 - accuracy: 0.8936
Epoch 3/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2886 - accuracy: 0.8981
Epoch 4/10
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2816 - accuracy: 0.8999
Epoch 5/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2741 - accuracy: 0.9041
Epoch 6/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2694 - accuracy: 0.9064
Epoch 7/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2629 - accuracy: 0.9086
Epoch 8/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2574 - accuracy: 0.9098
Epoch 9/10
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2538 - accuracy: 0.9115
Epoch 10/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.249

In [4]:
model.fit(train_scaled, train_target, epochs=20)

Epoch 1/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2435 - accuracy: 0.9153
Epoch 2/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2390 - accuracy: 0.9179
Epoch 3/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2334 - accuracy: 0.9190
Epoch 4/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2345 - accuracy: 0.9181
Epoch 5/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2257 - accuracy: 0.9234
Epoch 6/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2277 - accuracy: 0.9226
Epoch 7/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2217 - accuracy: 0.9251
Epoch 8/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2192 - accuracy: 0.9264
Epoch 9/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2181 - accuracy: 0.9279
Epoch 10/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.211

In [5]:
model.fit(train_scaled, train_target, epochs=40)

Epoch 1/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1832 - accuracy: 0.9399
Epoch 2/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1825 - accuracy: 0.9392
Epoch 3/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1806 - accuracy: 0.9420
Epoch 4/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1792 - accuracy: 0.9418
Epoch 5/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1781 - accuracy: 0.9419
Epoch 6/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1756 - accuracy: 0.9427
Epoch 7/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1757 - accuracy: 0.9437
Epoch 8/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.1742 - accuracy: 0.9456
Epoch 9/40
1500/1500 [==============================] - 4s 2ms/step - loss: 0.1701 - accuracy: 0.9443
Epoch 10/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.167

에포크를 높일수록 정확도가 향상됐습니다.

검증 세트에서의 성능도 확인해 보겠습니다.

In [10]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.9142 - accuracy: 0.8776


[0.9141854047775269, 0.8775833249092102]

## **옵티마이저**

위 `compile()` 메서드에서는 케라스의 기본 경사 하강법 알고리즘인 **RMSprop**(디폴트)를 사용했습니다. 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공합니다. 이 알고리즘들을 **옵티마이저(optimizer)**라고 부릅니다. 이번 파트에서는 여러 옵티마이저를 적용해 보겠습니다.

가장 기본적인 옵티마이저는 확률적 경사 하강법인 **SGD**입니다. 샘플 1개를 뽑아서 훈련하지 않고 미니배치를 사용합니다. SGD를 적용해 보겠습니다.

In [ ]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

학습률을 변경하려면 `SGD` 클래스의 인스턴스를 생성하여 매개변수에 지정합니다.

In [3]:
sgd = keras.optimizers.SGD(learning_rate=0.1)
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

기본 경사 하강법 옵티마이저 클래스는 아래와 같습니다. 이 옵티마이저 클래스에 매개변수를 추가로 지정하면 다른 옵티마이저가 됩니다. 많이 사용되는 옵티마이저 종류입니다.

- `SGD(learning_rate=0.01)`: 기본 경사 하강법입니다. 학습률(`learning_rate`) 디폴트는 `0.01`이며 따로 지정하지 않아도 됩니다.

 - `momentum`: 0보다 큰 값을 지정하면 **모멘텀** 옵티마이저입니다. 예컨대 `momentum=0.9` 방식입니다(보통 `0.9` 이상을 지정합니다). 기본 경사 하강법에서는 `0`(디폴트)입니다. 이 옵티마이저는 그레이디언트 가속도를 사용하는 **모멘텀 최적화(momentum optimization)**를 사용합니다.

 - `nesterov=True`: **네스테로프 모멘텀** 옵티마이저입니다. 기본 경사 하강법에서는 `False`(디폴트)입니다. **네스테로프 모멘텀 최적화(nesterov momentum optimization, 네스테로프 가속 경사)**를 사용합니다. 모멘텀 최적화를 2번 반복하여 구현하며 기본 확률적 경사 하강법보다 좋은 성능을 발휘합니다.

예시 코드는 아래와 같습니다.

In [ ]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

추가로 적응적 학습률 옵티마이저 또한 많이 사용됩니다. 모델이 최적점에 가까워질수록 학습률을 낮출 수 있고, 이를 통해 최적점에 안정적으로 수렴할 가능성이 높아집니다. 이 학습률을 **적응적 학습률(adaptive learning rate)**라고 부릅니다. 이 방식들은 학습률 매개변수를 튜닝하는 수고를 덜어줍니다. 적응적 학습률을 사용하는 대표적 옵티마이저는 **Adagrad**와 **RMSprop**입니다. `compile()` 메서드의 매개변수 `optimizer`에 각각 `'adagrad'`와 `'rmsprop'`로 지정하면 되지만 옵티마이저의 매개변수를 변경하려면 `Adagrad` 클래스와 `RMSprop` 클래스를 사용합니다. 목록으로 정리하자면 아래와 같습니다.

- `Adagrad`: Adagrad 옵티마이저 클래스입니다. 그레이디언트 제곱을 누적하여 학습률을 나눕니다.

 - `learning_rate`: 학습률을 지정합니다. 디폴트는 `0.001`입니다.

 - `initial_accumulator`: 그레이디언트 제곱에 대한 누적값의 초깃값을 지정합니다. 디폴트는 `0.1`입니다.

- `RMSprop`: RMSprop 옵티마이저 클래스입니다. Adagrad처럼 그레이디언트 제곱으로 학습률을 나누지만 최근의 그레이디언트를 사용하기 위해 지수 감소를 사용합니다.

 - `learning_rate`: 학습률을 지정합니다. 디폴트는 `0.001`입니다.

 - `rho`: 지수 감소 비율을 지정합니다. 디폴트는 `0.9`입니다.

In [ ]:
# 매개변수를 변경하지 않습니다.
model.compile(optimizer='adagrad', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics='accuracy')

# 매개변수를 변경합니다.
adagrad = keras.optimizers.Adagrad(learning=0.1)
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

rmsprop = keras.optimizers.RMSprop(learning=0.1)
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

**Adam**은 모멘텀 최적화와 RMSprop의 장점을 접목한 적응적 학습률 옵티마이저입니다. RMSprop처럼 가장 먼저 시도할 만한 알고리즘입니다. 사용법은 다른 옵티마이저와 같습니다.

- `Adam`: Adam 옵티마이저 클래스입니다.

 - `beta_1`: 모멘텀 최적화의 그레이디언트 지수 감소 평균을 조절합니다. 디폴트는 `0.9`입니다.

 - `beta_2`: RMSprop의 그레이디언트 제곱의 지수 감소 평균을 조절할 수 있습니다. 디폴트는 `0.999`입니다.

In [7]:
# 매개변수를 변경하지 않습니다.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

# 매개변수를 변경합니다.
adam = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics='accuracy')

Adam 클래스를 사용하여 모델을 훈련해 보겠습니다.

In [8]:
model = keras.Sequential(name='fashion_MNIST_model')
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu', name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5300 - accuracy: 0.8132
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3907 - accuracy: 0.8598
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3482 - accuracy: 0.8741
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3249 - accuracy: 0.8805
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3034 - accuracy: 0.8879


디폴트인 RMSprop를 사용했을 때와 비슷한 성능이 도출됐습니다.

검증 세트에서의 성능을 확인하겠습니다.

In [9]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3553 - accuracy: 0.8700


[0.35530829429626465, 0.8700000047683716]